In [1]:
import pandas as pd
import h5py



# Iemocap + previous utterance with speaker

In [2]:
iemo = pd.read_pickle('data/iemo.pkl')

In [3]:
names = {"Ses01_M": "James",
         "Ses02_M": "John",
         "Ses03_M": "Robert",
         "Ses04_M": "Michael",
         "Ses05_M": "William",
         "Ses01_F": "Mary" ,
         "Ses02_F": "Patricia",
         "Ses03_F": "Jennifer",
         "Ses04_F": "Linda",
         "Ses05_F": "Elizabeth",
}

iemo['Ses'] = iemo['Session'].apply(lambda x: x[:5] + x[-5:-3])
iemo['talker'] = iemo['Ses'].apply(lambda x: names[x].upper())
iemo['talker']

0            MARY
1           JAMES
2           JAMES
3           JAMES
4           JAMES
          ...    
7375      WILLIAM
7376      WILLIAM
7377      WILLIAM
7378      WILLIAM
7379    ELIZABETH
Name: talker, Length: 7380, dtype: object

In [4]:
iemo['text_w_speaker'] = iemo['talker'] + ':' + iemo['text']
iemo['text_w_speaker']

0                           MARY: What's he going to say?
1                                       JAMES: He saw it.
2                   JAMES: He was out here when it broke.
3       JAMES: Four o clock this morning.  I heard a c...
4                                    JAMES: I don't know.
                              ...                        
7375    WILLIAM: [LAUGHTER] That was quit a rouser, wa...
7376             WILLIAM: Oh, I didn't hit you very hard.
7377    WILLIAM: I should never forget his face. [LAUG...
7378             WILLIAM: We were very much younger then.
7379       ELIZABETH: You knew there was nothing in that.
Name: text_w_speaker, Length: 7380, dtype: object

In [5]:
def concatenate_utterances(df):
    df['concatenated'] = ''
    groups = df.groupby(['split', 'dialog'])
    for _, group in groups:
        utterances = group['text_w_speaker'].tolist()
        for i in range(len(utterances)):
            if (i == 0):
                concatenated = utterances[i]
            else:
                concatenated = utterances[i-1] + ' </s></s> ' + utterances[i]
            df.loc[group.index[i], 'concatenated'] = concatenated
    return df

iemo = concatenate_utterances(iemo)
iemo

,text,Session,timings,audio_path,video_path,emotion_label,"V,A,D",emotion,speaker,sentiment,...,split,bbox,face_bbox,len,filename,dialog,Ses,talker,text_w_speaker,concatenated
0,What's he going to say?,Ses01F_script01_1_F000,"(6.21, 9.32)",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,frustrated,"[2.0000, 2.3333, 2.3333]",0,True,0,...,train,"[[[65, 33, 200, 217]], [[63, 32, 203, 217]], [...","[[[108, 50, 133, 86]], [[108, 50, 133, 85]], [...",16,Ses01F_script01_1,7,Ses01_F,MARY,MARY: What's he going to say?,MARY: What's he going to say?
1,He saw it.,Ses01F_script01_1_M000,"(11.9892, 15.1219)",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,neutral,"[3.0000, 1.6667, 2.3333]",1,False,1,...,train,"[[[24, 37, 172, 217], [0, 86, 36, 219], [182, ...","[[[111, 57, 140, 101], [13, 96, 25, 113]], [[1...",16,Ses01F_script01_1,7,Ses01_M,JAMES,JAMES: He saw it.,MARY: What's he going to say? </s></s> JAMES: ...
2,He was out here when it broke.,Ses01F_script01_1_M001,"(19.0456, 22.3962)",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,neutral,"[3.0000, 2.0000, 2.3333]",1,False,1,...,train,"[[[19, 36, 175, 216], [2, 62, 54, 224], [182, ...","[[[105, 67, 132, 115], [21, 82, 35, 103]], [[1...",16,Ses01F_script01_1,7,Ses01_M,JAMES,JAMES: He was out here when it broke.,JAMES: He saw it. </s></s> JAMES: He was out h...
3,Four o clock this morning. I heard a crack a...,Ses01F_script01_1_M002,"(24.8, 35.9971)",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,neutral,"[2.6667, 3.0000, 2.3333]",1,False,1,...,train,"[[[40, 37, 176, 217], [2, 74, 68, 220], [183, ...","[[[115, 66, 145, 113], [21, 83, 35, 105]], [[1...",16,Ses01F_script01_1,7,Ses01_M,JAMES,JAMES: Four o clock this morning. I heard a c...,JAMES: He was out here when it broke. </s></s>...
4,I don't know.,Ses01F_script01_1_M003,"(38.66, 40.53)",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,neutral,"[3.0000, 2.0000, 2.0000]",1,False,1,...,train,"[[[27, 35, 168, 218], [184, 129, 222, 205], [0...","[[[113, 51, 141, 98]], [[112, 51, 140, 98]], [...",16,Ses01F_script01_1,7,Ses01_M,JAMES,JAMES: I don't know.,JAMES: Four o clock this morning. I heard a c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7375,"[LAUGHTER] That was quit a rouser, wasn't it?",Ses05M_script03_1_M030,"(215.4, 220.75)",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,excited,"[4.5000, 4.5000, 3.5000]",4,True,4,...,val,"[[[61, 22, 188, 216]], [[60, 20, 187, 215]], [...","[[[119, 39, 151, 81]], [[119, 40, 148, 82]], [...",16,Ses05M_script03_1,149,Ses05_M,WILLIAM,"WILLIAM: [LAUGHTER] That was quit a rouser, wa...","ELIZABETH: Oh, It burnt my comb and all the to..."
7376,"Oh, I didn't hit you very hard.",Ses05M_script03_1_M031,"(221.13, 223.83)",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,excited,"[2.5000, 3.5000, 3.5000]",4,True,4,...,val,"[[[61, 16, 189, 216]], [[61, 15, 188, 216]], [...","[[[129, 34, 157, 73]], [[126, 32, 157, 76]], [...",16,Ses05M_script03_1,149,Ses05_M,WILLIAM,"WILLIAM: Oh, I didn't hit you very hard.","WILLIAM: [LAUGHTER] That was quit a rouser, wa..."
7377,I should never forget his face. [LAUGHTER],Ses05M_script03_1_M032,"(229.88, 233.06)",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,excited,"[4.5000, 4.5000, 4.5000]",4,True,4,...,val,"[[[57, 13, 190, 213]], [[56, 16, 190, 215]], [...","[[[105, 30, 133, 69]], [[110, 37, 140, 78]], [...",16,Ses05M_script03_1,149,Ses05_M,WILLIAM,WILLIAM: I should never forget his face. [LAUG...,"WILLIAM: Oh, I didn't hit you very hard. </s><..."
7378,We were very much younger then.,Ses05M_script03_1_M033,"(234.14, 236.66)",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/S

In [6]:
iemo.rename(columns={'text_w_speaker': 'og_text_with_auth','text': 'og_text' , 'concatenated' : 'text'}, inplace=True)
iemo

,og_text,Session,timings,audio_path,video_path,emotion_label,"V,A,D",emotion,speaker,sentiment,...,split,bbox,face_bbox,len,filename,dialog,Ses,talker,og_text_with_auth,text
0,What's he going to say?,Ses01F_script01_1_F000,"(6.21, 9.32)",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,frustrated,"[2.0000, 2.3333, 2.3333]",0,True,0,...,train,"[[[65, 33, 200, 217]], [[63, 32, 203, 217]], [...","[[[108, 50, 133, 86]], [[108, 50, 133, 85]], [...",16,Ses01F_script01_1,7,Ses01_F,MARY,MARY: What's he going to say?,MARY: What's he going to say?
1,He saw it.,Ses01F_script01_1_M000,"(11.9892, 15.1219)",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,neutral,"[3.0000, 1.6667, 2.3333]",1,False,1,...,train,"[[[24, 37, 172, 217], [0, 86, 36, 219], [182, ...","[[[111, 57, 140, 101], [13, 96, 25, 113]], [[1...",16,Ses01F_script01_1,7,Ses01_M,JAMES,JAMES: He saw it.,MARY: What's he going to say? </s></s> JAMES: ...
2,He was out here when it broke.,Ses01F_script01_1_M001,"(19.0456, 22.3962)",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,neutral,"[3.0000, 2.0000, 2.3333]",1,False,1,...,train,"[[[19, 36, 175, 216], [2, 62, 54, 224], [182, ...","[[[105, 67, 132, 115], [21, 82, 35, 103]], [[1...",16,Ses01F_script01_1,7,Ses01_M,JAMES,JAMES: He was out here when it broke.,JAMES: He saw it. </s></s> JAMES: He was out h...
3,Four o clock this morning. I heard a crack a...,Ses01F_script01_1_M002,"(24.8, 35.9971)",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,neutral,"[2.6667, 3.0000, 2.3333]",1,False,1,...,train,"[[[40, 37, 176, 217], [2, 74, 68, 220], [183, ...","[[[115, 66, 145, 113], [21, 83, 35, 105]], [[1...",16,Ses01F_script01_1,7,Ses01_M,JAMES,JAMES: Four o clock this morning. I heard a c...,JAMES: He was out here when it broke. </s></s>...
4,I don't know.,Ses01F_script01_1_M003,"(38.66, 40.53)",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,neutral,"[3.0000, 2.0000, 2.0000]",1,False,1,...,train,"[[[27, 35, 168, 218], [184, 129, 222, 205], [0...","[[[113, 51, 141, 98]], [[112, 51, 140, 98]], [...",16,Ses01F_script01_1,7,Ses01_M,JAMES,JAMES: I don't know.,JAMES: Four o clock this morning. I heard a c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7375,"[LAUGHTER] That was quit a rouser, wasn't it?",Ses05M_script03_1_M030,"(215.4, 220.75)",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,excited,"[4.5000, 4.5000, 3.5000]",4,True,4,...,val,"[[[61, 22, 188, 216]], [[60, 20, 187, 215]], [...","[[[119, 39, 151, 81]], [[119, 40, 148, 82]], [...",16,Ses05M_script03_1,149,Ses05_M,WILLIAM,"WILLIAM: [LAUGHTER] That was quit a rouser, wa...","ELIZABETH: Oh, It burnt my comb and all the to..."
7376,"Oh, I didn't hit you very hard.",Ses05M_script03_1_M031,"(221.13, 223.83)",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,excited,"[2.5000, 3.5000, 3.5000]",4,True,4,...,val,"[[[61, 16, 189, 216]], [[61, 15, 188, 216]], [...","[[[129, 34, 157, 73]], [[126, 32, 157, 76]], [...",16,Ses05M_script03_1,149,Ses05_M,WILLIAM,"WILLIAM: Oh, I didn't hit you very hard.","WILLIAM: [LAUGHTER] That was quit a rouser, wa..."
7377,I should never forget his face. [LAUGHTER],Ses05M_script03_1_M032,"(229.88, 233.06)",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,excited,"[4.5000, 4.5000, 4.5000]",4,True,4,...,val,"[[[57, 13, 190, 213]], [[56, 16, 190, 215]], [...","[[[105, 30, 133, 69]], [[110, 37, 140, 78]], [...",16,Ses05M_script03_1,149,Ses05_M,WILLIAM,WILLIAM: I should never forget his face. [LAUG...,"WILLIAM: Oh, I didn't hit you very hard. </s><..."
7378,We were very much younger then.,Ses05M_script03_1_M033,"(234.14, 236.66)",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses

# Concat Df's

In [7]:
meld = pd.read_pickle('data/meld.pkl')
meld.columns

Index(['talker', 'emotion_label', 'sentiment_label', 'dialog', 'utterance',
       'split', 'name', 'text', 'emotion', 'sentiment', 'timings', 'bbox',
       'audio_path', 'video_path', 'context_speaker', 'context_name',
       'og_text', 'len', 'og_text_with_auth', 'concatenated_with_auth',
       'concatenated'],
      dtype='object')

In [8]:
meld_drop = list(set(meld.columns.to_list()) - set(iemo.columns.to_list()))
meld_drop

['name',
 'context_name',
 'concatenated_with_auth',
 'concatenated',
 'context_speaker',
 'utterance']

In [9]:
iemo_drop = list(set(iemo.columns.to_list()) - set(meld.columns.to_list()))
iemo_drop

['speaker', 'filename', 'face_bbox', 'Ses', 'Session', 'V,A,D']

In [10]:
small_meld = meld.drop(meld_drop, axis=1)
small_iemo = iemo.drop(iemo_drop, axis=1)

In [11]:
small_iemo.columns, small_meld.columns

(Index(['og_text', 'timings', 'audio_path', 'video_path', 'emotion_label',
        'emotion', 'sentiment', 'sentiment_label', 'split', 'bbox', 'len',
        'dialog', 'talker', 'og_text_with_auth', 'text'],
       dtype='object'),
 Index(['talker', 'emotion_label', 'sentiment_label', 'dialog', 'split', 'text',
        'emotion', 'sentiment', 'timings', 'bbox', 'audio_path', 'video_path',
        'og_text', 'len', 'og_text_with_auth'],
       dtype='object'))

In [40]:
df = pd.concat([small_meld, small_iemo], ignore_index=True)
df

,talker,emotion_label,sentiment_label,dialog,split,text,emotion,sentiment,timings,bbox,audio_path,video_path,og_text,len,og_text_with_auth
0,Mark,surprise,positive,0,test,MARK: Why do all you’re coffee mugs have numbe...,2,2,"(0.08076116071428571, 2.0190290178571426)","[[[67, 44, 111, 224], [117, 24, 165, 223]], [[...",../../data/test_splits_wav/dia0_utt0.wav,../../data/test_video/dia0_utt0.mp4,Why do all you’re coffee mugs have numbers on...,193,MARK: Why do all you’re coffee mugs have numbe...
1,Rachel,anger,negative,0,test,MARK: Why do all you’re coffee mugs have numbe...,3,1,"(0.30100982937685455, 5.638917470326409)","[[[101, 25, 160, 226], [27, 57, 91, 221]], [[9...",../../data/test_splits_wav/dia0_utt1.wav,../../data/test_video/dia0_utt1.mp4,Oh. That’s so Monica can keep track. That way...,193,RACHEL: Oh. That’s so Monica can keep track. T...
2,Rachel,neutral,neutral,0,test,RACHEL: Oh. That’s so Monica can keep track. T...,0,0,"(0.525967672413793, 1.0114762931034482)","[[[38, 44, 127, 223], [125, 65, 163, 184]], [[...",../../data/test_splits_wav/dia0_utt2.wav,../../data/test_video/dia0_utt2.mp4,Y'know what?,193,RACHEL: Y'know what?
3,Joey,neutral,neutral,1,test,"JOEY: Come on, Lydia, you can do it.",0,0,"(0.020053125, 1.7646750000000002)","[[[106, 76, 190, 216], [50, 47, 107, 133]], [[...",../../data/test_splits_wav/dia1_utt0.wav,../../data/test_video/dia1_utt0.mp4,"Come on, Lydia, you can do it.",193,"JOEY: Come on, Lydia, you can do it."
4,Joey,joy,positive,1,test,"JOEY: Come on, Lydia, you can do it. </s></s> ...",1,2,"(0.3516875, 0.4344375)","[[[51, 38, 92, 122], [76, 80, 190, 208]], [[51...",../../data/test_splits_wav/dia1_utt1.wav,../../data/test_video/dia1_utt1.mp4,Push!,193,JOEY: Push!
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21079,WILLIAM,excited,excited,149,val,"ELIZABETH: Oh, It burnt my comb and all the to...",4,4,"(215.4, 220.75)","[[[61, 22, 188, 216]], [[60, 20, 187, 215]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,"[LAUGHTER] That was quit a rouser, wasn't it?",16,"WILLIAM: [LAUGHTER] That was quit a rouser, wa..."
21080,WILLIAM,excited,excited,149,val,"WILLIAM: [LAUGHTER] That was quit a rouser, wa...",4,4,"(221.13, 223.83)","[[[61, 16, 189, 216]], [[61, 15, 188, 216]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,"Oh, I didn't hit you very hard.",16,"WILLIAM: Oh, I didn't hit you very hard."
21081,WILLIAM,excited,excited,149,val,"WILLIAM: Oh, I didn't hit you very hard. </s><...",4,4,"(229.88, 233.06)","[[[57, 13, 190, 213]], [[56, 16, 190, 215]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,I should never forget his face. [LAUGHTER],16,WILLIAM: I should never forget his face. [LAUG...
21082,WILLIAM,excited,excited,149,val,WILLIAM: I should never forget his face. [LAUG...,4,4,"(234.14, 236.66)","[[[62, 11, 190, 215]], [[60, 13, 189, 214]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,We were very much younger then.,16,WILLIAM: We were very much younger then.


# Mapping

In [41]:
df[pd.isna(df['emotion'])]  , df[pd.isna(df['emotion_label'])]

(Empty DataFrame
 Columns: [talker, emotion_label, sentiment_label, dialog, split, text, emotion, sentiment, timings, bbox, audio_path, video_path, og_text, len, og_text_with_auth]
 Index: [],
 Empty DataFrame
 Columns: [talker, emotion_label, sentiment_label, dialog, split, text, emotion, sentiment, timings, bbox, audio_path, video_path, og_text, len, og_text_with_auth]
 Index: [])

In [43]:
emotion_mapping = {
    
    
    "happy": "joy",
    "excited": "joy",
    "angry": "anger",
    "frustrated": "anger",
    "sad": "sadness",
    
    
    
    
    "neutral" : "neutral",
    "joy" : "joy",
    "surprise" : "surprise",
    "anger" : "anger",
    "sadness" : "sadness",
    "disgust" : "disgust",
    "fear" : "fear",
}

emotion2id = {
    "neutral": 0,
    "joy": 1,
    "surprise": 2,
    "anger": 3,
    "sadness": 4,
    "disgust": 5,
    "fear": 6,
}



In [45]:
df['emotion_label'] = df['emotion_label'].map(emotion_mapping)

# Replace the 'label' column with their emotion2id value
df['emotion'] = df['emotion_label'].map(emotion2id)


In [46]:
df[pd.isna(df['emotion'])]  , df[pd.isna(df['emotion_label'])]

(Empty DataFrame
 Columns: [talker, emotion_label, sentiment_label, dialog, split, text, emotion, sentiment, timings, bbox, audio_path, video_path, og_text, len, og_text_with_auth]
 Index: [],
 Empty DataFrame
 Columns: [talker, emotion_label, sentiment_label, dialog, split, text, emotion, sentiment, timings, bbox, audio_path, video_path, og_text, len, og_text_with_auth]
 Index: [])

In [47]:
df

,talker,emotion_label,sentiment_label,dialog,split,text,emotion,sentiment,timings,bbox,audio_path,video_path,og_text,len,og_text_with_auth
0,Mark,surprise,positive,0,test,MARK: Why do all you’re coffee mugs have numbe...,2,2,"(0.08076116071428571, 2.0190290178571426)","[[[67, 44, 111, 224], [117, 24, 165, 223]], [[...",../../data/test_splits_wav/dia0_utt0.wav,../../data/test_video/dia0_utt0.mp4,Why do all you’re coffee mugs have numbers on...,193,MARK: Why do all you’re coffee mugs have numbe...
1,Rachel,anger,negative,0,test,MARK: Why do all you’re coffee mugs have numbe...,3,1,"(0.30100982937685455, 5.638917470326409)","[[[101, 25, 160, 226], [27, 57, 91, 221]], [[9...",../../data/test_splits_wav/dia0_utt1.wav,../../data/test_video/dia0_utt1.mp4,Oh. That’s so Monica can keep track. That way...,193,RACHEL: Oh. That’s so Monica can keep track. T...
2,Rachel,neutral,neutral,0,test,RACHEL: Oh. That’s so Monica can keep track. T...,0,0,"(0.525967672413793, 1.0114762931034482)","[[[38, 44, 127, 223], [125, 65, 163, 184]], [[...",../../data/test_splits_wav/dia0_utt2.wav,../../data/test_video/dia0_utt2.mp4,Y'know what?,193,RACHEL: Y'know what?
3,Joey,neutral,neutral,1,test,"JOEY: Come on, Lydia, you can do it.",0,0,"(0.020053125, 1.7646750000000002)","[[[106, 76, 190, 216], [50, 47, 107, 133]], [[...",../../data/test_splits_wav/dia1_utt0.wav,../../data/test_video/dia1_utt0.mp4,"Come on, Lydia, you can do it.",193,"JOEY: Come on, Lydia, you can do it."
4,Joey,joy,positive,1,test,"JOEY: Come on, Lydia, you can do it. </s></s> ...",1,2,"(0.3516875, 0.4344375)","[[[51, 38, 92, 122], [76, 80, 190, 208]], [[51...",../../data/test_splits_wav/dia1_utt1.wav,../../data/test_video/dia1_utt1.mp4,Push!,193,JOEY: Push!
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21079,WILLIAM,joy,excited,149,val,"ELIZABETH: Oh, It burnt my comb and all the to...",1,4,"(215.4, 220.75)","[[[61, 22, 188, 216]], [[60, 20, 187, 215]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,"[LAUGHTER] That was quit a rouser, wasn't it?",16,"WILLIAM: [LAUGHTER] That was quit a rouser, wa..."
21080,WILLIAM,joy,excited,149,val,"WILLIAM: [LAUGHTER] That was quit a rouser, wa...",1,4,"(221.13, 223.83)","[[[61, 16, 189, 216]], [[61, 15, 188, 216]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,"Oh, I didn't hit you very hard.",16,"WILLIAM: Oh, I didn't hit you very hard."
21081,WILLIAM,joy,excited,149,val,"WILLIAM: Oh, I didn't hit you very hard. </s><...",1,4,"(229.88, 233.06)","[[[57, 13, 190, 213]], [[56, 16, 190, 215]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,I should never forget his face. [LAUGHTER],16,WILLIAM: I should never forget his face. [LAUG...
21082,WILLIAM,joy,excited,149,val,WILLIAM: I should never forget his face. [LAUG...,1,4,"(234.14, 236.66)","[[[62, 11, 190, 215]], [[60, 13, 189, 214]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,We were very much younger then.,16,WILLIAM: We were very much younger then.


In [49]:
df['emotion_label'].value_counts().sum()

21084

In [48]:
df.to_pickle('data/meld_iemo.pkl')

# H5PY Join

In [ ]:
def join_hdf5(file1, file2, output_file):
    with h5py.File(file1, 'r' , libver="latest", swmr=True) as f1, h5py.File(file2, 'r' , libver="latest", swmr=True) as f2, h5py.File(output_file, 'w' , libver="latest", swmr=True) as fo:
        # Iterate over datasets in the first file
        for name, data in f1.items():
            # Copy each dataset to the output file
            f1.copy(name, fo)

        # Iterate over datasets in the second file
        for name, data in f2.items():
            # Copy each dataset to the output file
            f2.copy(name, fo)

    print(f"Joined hdf5 files saved to {output_file}")



In [20]:

def video():
    vf1 = 'meld_videos_blackground.hdf5'
    vf2 = 'iemo_videos_blackground.hdf5'
    output_vf = 'meld_iemo_videos_blackground.hdf5'

    with h5py.File(vf1, 'r' , libver="latest", swmr=True) as f1, h5py.File(vf2, 'r' , libver="latest", swmr=True) as f2, h5py.File(output_vf, 'w' , libver="latest", swmr=True) as fo:
        # Iterate over datasets in the first file
        for name, data in f1.items():
            # Copy each dataset to the output file
            f1.copy(name, fo)

        # Iterate over datasets in the second file
        for name, data in f2.items():
            # Copy each dataset to the output file
            f2.copy(name, fo)

def audio():
    af1 = 'whisper_meld_audio.hdf5'
    af2 = 'whisper_iemo_audio.hdf5'
    output_af = 'whisper_meld_iemo_audio.hdf5'

    with h5py.File(af1, 'r' , libver="latest", swmr=True) as f1, h5py.File(af2, 'r' , libver="latest", swmr=True) as f2, h5py.File(output_af, 'w' , libver="latest", swmr=True) as fo:
        # Iterate over datasets in the first file
        for name, data in f1.items():
            # Copy each dataset to the output file
            f1.copy(name, fo)

        # Iterate over datasets in the second file
        for name, data in f2.items():
            # Copy each dataset to the output file
            f2.copy(name, fo)

/var/folders/3c/w5kd6n2n7x5dr7k30ktsk6gh0000gn/T/ipykernel_46678/647726562.py:5: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  with h5py.File(file1, 'r' , libver="latest", swmr=True) as f1, h5py.File(file2, 'r' , libver="latest", swmr=True) as f2, h5py.File(output_file, 'w' , libver="latest", swmr=True) as fo:


In [22]:
r_f = h5py.File(output_file, 'r' , libver="latest", swmr=True)

In [23]:
r_f.keys()

<KeysViewHDF5 ['test_dia0_utt0_(0.08076116071428571, 2.0190290178571426)', 'train_Ses01F_script01_1_(6.21, 9.32)']>

In [24]:
r_f['test_dia0_utt0_(0.08076116071428571, 2.0190290178571426)'][()]

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    

# Delete Lter

In [61]:
meld_iemo = pd.read_pickle('data/meld_iemo.pkl')
# dia101 = meld[meld['dialog'] == 1019]
# dia101_utt5 = dia101[dia101['utterance'] == 5]
# dia101_utt5.drop(columns=['text', 'emotion', 'emotion_label', 'split',  'og_text_with_auth', 'og_text'])

In [64]:
small_wav = meld_iemo[pd.isna(meld_iemo['timings'])]['audio_path'].to_list()

In [68]:
"../../data/train_splits_wav/dia354_utt0.wav" in small_wav

False

In [66]:
small_wav

['../../data/test_splits_wav/dia25_utt15.wav',
 '../../data/test_splits_wav/dia35_utt1.wav',
 '../../data/test_splits_wav/dia48_utt6.wav',
 '../../data/test_splits_wav/dia54_utt6.wav',
 '../../data/test_splits_wav/dia72_utt4.wav',
 '../../data/test_splits_wav/dia80_utt1.wav',
 '../../data/test_splits_wav/dia83_utt3.wav',
 '../../data/test_splits_wav/dia88_utt12.wav',
 '../../data/test_splits_wav/dia95_utt2.wav',
 '../../data/test_splits_wav/dia95_utt12.wav',
 '../../data/test_splits_wav/dia96_utt6.wav',
 '../../data/test_splits_wav/dia116_utt4.wav',
 '../../data/test_splits_wav/dia124_utt8.wav',
 '../../data/test_splits_wav/dia125_utt21.wav',
 '../../data/test_splits_wav/dia126_utt11.wav',
 '../../data/test_splits_wav/dia126_utt15.wav',
 '../../data/test_splits_wav/dia132_utt17.wav',
 '../../data/test_splits_wav/dia147_utt3.wav',
 '../../data/test_splits_wav/dia151_utt3.wav',
 '../../data/test_splits_wav/dia155_utt4.wav',
 '../../data/test_splits_wav/dia160_utt4.wav',
 '../../data/test